In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('/Users/ISRAEL/Downloads/energydata_complete.csv')
df.head(2)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.7900,19.79,44.73,19.0,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.7225,19.79,44.79,19.0,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195


In [3]:
# Renaming columns

column_names={'T1':'TempKitchen_area', 'RH_1':'HumidityKitchen_area', 'T2':'TempLiving_area', 'RH_2':'HumidityLiving_area', 'T3':'TempLaundry_room_area', 'RH_3':'HumidityLaundry_room_area', 'T4':'TempOffice_room', 'RH_4':'HumidityOffice_room', 'T5':'TempBath_room', 'RH_5':'HumidityBath_room', 'T6':'TempOutside_building', 'RH_6':'HumidityOutside_building', 'T7':'TempIroning_room', 'RH_7':'HumidityIroning_room', 'T8':'TempTeenager_room', 'RH_8':'HumidityTeenager_room', 'T9':'TempParents_room', 'RH_9':'HumidityParents_room'}
df = df.rename(columns=column_names)
df.head(2)

,date,Appliances,lights,TempKitchen_area,HumidityKitchen_area,TempLiving_area,HumidityLiving_area,TempLaundry_room_area,HumidityLaundry_room_area,TempOffice_room,...,TempParents_room,HumidityParents_room,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.7900,19.79,44.73,19.0,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.7225,19.79,44.79,19.0,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195


In [4]:
# Question 12

df_12 = df[['TempLiving_area', 'TempOutside_building']]

from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(df_12[['TempLiving_area']], df_12[['TempOutside_building']])

predicted = model.predict(df_12[['TempLiving_area']])


from sklearn.metrics import r2_score

Rsquared = r2_score(df_12[['TempOutside_building']], predicted)

print(round(Rsquared, 2))

0.64


In [5]:
# Getting required columns
df_dropped = df.drop(columns=['date', 'lights'])

# Importing MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled = pd.DataFrame(scaler.fit_transform(df_dropped), columns=df_dropped.columns)

# Getting the independent and dependent variables
predictors = scaled.drop(columns=['Appliances'])
target = scaled['Appliances']

# Splitting the data
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

# Fitting into the model
linear_model = LinearRegression()

linear_model.fit(x_train, y_train)

predicted_values = linear_model.predict(x_test)

print(predicted_values)

[0.03322207 0.24411599 0.03400024 ... 0.06844707 0.10032325 0.05722198]


In [6]:
# Question 13

from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, predicted_values)
print(round(mae, 2))

0.05


In [7]:
# Question 14

rss = np.sum(np.square(y_test - predicted_values))
print(round(rss, 2))

45.35


In [8]:
# Question 15

from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, predicted_values))

print(round(rmse, 3))

0.088


In [9]:
# Question 16

from sklearn.metrics import r2_score

rsquared =  r2_score(y_test, predicted_values)
print(round(rsquared, 2))

0.15


In [10]:
# Question 17

def get_weights(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

lmw = get_weights(linear_model, x_train, 'Linear_Model_Weight')

print(lmw[(lmw['Linear_Model_Weight'] == lmw['Linear_Model_Weight'].min())])
print(lmw[(lmw['Linear_Model_Weight'] == lmw['Linear_Model_Weight'].max())])


              Features  Linear_Model_Weight
0  HumidityLiving_area            -0.456698
                Features  Linear_Model_Weight
25  HumidityKitchen_area             0.553547


In [11]:
# Question 18

from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)

predicted_ridge = ridge_reg.predict(x_test)

rmse_ridge = np.sqrt(mean_squared_error(y_test, predicted_ridge))
print(round(rmse_ridge, 3))

0.088


In [18]:
# Question 19

from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

predicted_lasso = lasso_reg.predict(x_test)
print(predicted_lasso)

def get_weights(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

lmw_lasso = get_weights(lasso_reg, x_train, 'Linear_Model_Weight')

print(lmw_lasso)

[0.07370267 0.08143458 0.07716072 ... 0.07792848 0.09034412 0.08359255]
                     Features  Linear_Model_Weight
0                      RH_out            -0.049557
1       HumidityTeenager_room            -0.000110
2            TempKitchen_area             0.000000
3                   Tdewpoint             0.000000
4                  Visibility             0.000000
5                 Press_mm_hg            -0.000000
6                       T_out             0.000000
7        HumidityParents_room            -0.000000
8            TempParents_room            -0.000000
9           TempTeenager_room             0.000000
10       HumidityIroning_room            -0.000000
11                        rv1            -0.000000
12           TempIroning_room            -0.000000
13       TempOutside_building             0.000000
14          HumidityBath_room             0.000000
15              TempBath_room            -0.000000
16        HumidityOffice_room             0.000000
17        

In [19]:
# Question 20

rmse_lasso = np.sqrt(mean_squared_error(y_test, predicted_lasso))
print(round(rmse_lasso, 3))

0.094
